### Centering


Center using translation each subject MRI to his/her T2 scan. During training, we randomly introduce affine augmentation to the dataset. This preprocessing step ensure that the brain stays within the volume given the affine augmentation we introduce. It also helps during the pretraining step of our algorithm. 

The script below creates a folder `../data/centered_IXI/` containining 6 folders `T1`, `T2`, `PD`, `T1_mask`, , `T2_mask` and `PD_mask`. Each brain has the following naming convention: subject `../data/centered_IXI/T1/IXI261-HH-1704.nii` will have a corresponding mask `../data/centered_IXI/T1_mask/IXI261-HH-1704_mask.nii.gz` and so on...

In [3]:
import os
import ants
import SimpleITK as sitk
from pathlib import Path
from tqdm.notebook import tqdm

PATH = '../data/processed_IXI/'
SAVED_PATH = '../data/centered_IXI/'
Path(SAVED_PATH).mkdir(parents=True, exist_ok=True)
Path(SAVED_PATH+'/T1/').mkdir(parents=True, exist_ok=True)
Path(SAVED_PATH+'/T1_mask/').mkdir(parents=True, exist_ok=True)
Path(SAVED_PATH+'/T2/').mkdir(parents=True, exist_ok=True)
Path(SAVED_PATH+'/T2_mask/').mkdir(parents=True, exist_ok=True)
Path(SAVED_PATH+'/PD/').mkdir(parents=True, exist_ok=True)
Path(SAVED_PATH+'/PD_mask/').mkdir(parents=True, exist_ok=True)

directories = os.listdir(PATH+'T2/')

for mod in ['T1', 'PD', 'T2']:
    print('Processing {}'.format(mod))
    for i, d in tqdm(enumerate(directories)):
        if 'ipynb_checkpoints' in d:
            continue
            
        """STATIC"""
        itk = sitk.ReadImage(PATH+'T2/'+d)
        target = sitk.GetArrayFromImage(itk)
        itk = sitk.ReadImage(PATH+'T2_mask/'+d[:-4]+'_mask.nii.gz')
        target_mask = sitk.GetArrayFromImage(itk)        
        static = ants.from_numpy(target*target_mask)    
        
        if mod == 'T2':
            itkimage = sitk.GetImageFromArray(target.astype('float32'))
            sitk.WriteImage(itkimage, 
                            SAVED_PATH+'/{}/'.format(mod)+d)
            itkimage = sitk.GetImageFromArray(target_mask.astype('float32'))
            sitk.WriteImage(itkimage, 
                            SAVED_PATH+'/{}_mask/'.format(mod)+d[:-4]+'_mask.nii.gz')            
            continue
                 
        """MOVING"""
        itk = sitk.ReadImage(PATH+'{}/'.format(mod)+d)
        mr = sitk.GetArrayFromImage(itk)

        itk = sitk.ReadImage(PATH+'{}_mask/'.format(mod)+d[:-4]+'_mask.nii.gz')
        mask = sitk.GetArrayFromImage(itk)

        moving = ants.from_numpy(mr*mask)
        moving_mr = ants.from_numpy(mr)
        moving_mask = ants.from_numpy(mask.astype('float64'))

        mytx = ants.registration(fixed=static,
                                 moving=moving, 
                                 type_of_transform ='Translation',
                                 verbose=False)

        aligned_mr = ants.apply_transforms(fixed=static,
                                           moving=moving_mr,
                                           interpolator='linear',
                                           transformlist=mytx['fwdtransforms']).numpy().astype('float32')

        aligned_mask = ants.apply_transforms(fixed=static,
                                            moving=moving_mask,
                                            interpolator='genericLabel',
                                            transformlist=mytx['fwdtransforms']).numpy().astype('int32')


        itkimage = sitk.GetImageFromArray(aligned_mr)
        sitk.WriteImage(itkimage, 
                        SAVED_PATH+'/{}/'.format(mod)+d)

        itkimage = sitk.GetImageFromArray(aligned_mask)
        sitk.WriteImage(itkimage, 
                        SAVED_PATH+'/{}_mask/'.format(mod)+d[:-4]+'_mask.nii.gz')

Processing T1


0it [00:00, ?it/s]

AttributeError: module 'ants' has no attribute 'from_numpy'